# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [23]:
#Получим данные, импортируем библиотеки
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
#Прочтем данные и посмотрим на них
data = pd.read_csv('/datasets/data.csv')
display(data.head(10))
data.info()
display(data['days_employed'].isnull().sum())
display(data['total_income'].isnull().sum())
display(data[data.isna().any(axis=1)])
data.describe()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


2174

2174

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Общая информация:
children — количество детей в семье
days_employed — трудовой стаж в днях
dob_days — возраст клиента в годах
education — образование клиента
education_id — идентификатор образования
family_status — семейное положение
family_status_id — идентификатор семейного положения
gender — пол клиента
income_type — тип занятости
debt — имел ли задолженность по возврату кредитов
total_income — доход в месяц
purpose — цель получения кредита
При вызове метода describe(), который используется для просмотра некоторых основных статистических данных сразу видны некоторые необъяснимые значения. Начнем с начала: 
Количество детей в семье: минимальное значение отрицательно, что невозможно. Трудовой стаж днях уже из первой таблице видно, что отрицателен, что невозможно. Странное значение среднего трудового стажа в днях, слишком большое. В столбце возраста клиента в годах минимальное значение - 0, что тоже скорее всего является ошибкой.
С помощью метода info() - точно оказалось, что отсутствуют значения только в столбцах 'days_employed' и 'total_income'
C использование isna () для выбора всех строк с NaN было выявленно, что пропущенно в столбцах 'days_employed' и 'total_income',  при чем одинаковые значения это одни и те же строки, где отсутствуют данные по обоим столбцам. Возможно, эти люди никогда не работали, так как молоды или работали не официально. Ну либо данные некорректно выгружены. 
Проблемы, которые нужно решить:
- отрицательные значения в детях (а также значение 20 детей возможно является неправильным)
- отрицательные значения в днях стажа 
- отсутствую строки в днях стажа и доходе
- некорректные типы данных — в днях стажа и в доходе плавающая точка
- неудобные данные для анализа — дни стажа лучше перевести в года
- прыгающий регистр букв в образовании
- есть дублирующие признаки — избыточны образование и его идентификатор и такая же проблема у семейного положения
- минимальное значение в возрасте — 0

## Шаг 2. Предобработка данных

### Обработка пропусков

In [24]:
#Разбирая столбец трудового стажа видно, что огромные числа все положительные
#Поэтому положительные значения необходимо поделить на 24
display(data.head(50))
data.loc[data['days_employed'] > 0, 'days_employed'] = data['days_employed']/24
display(data.head(50))
#Разберемся с отрицательными значениями
data['days_employed'] = data['days_employed'].abs()
data['total_income'] = data['total_income'].abs()

print(data['income_type'].unique())
#Значение медианы
median = data.groupby('income_type').agg({'total_income' : 'median'})
 
for element in data['income_type'].unique():
    data.loc[data['income_type'] == element, 'total_income'] = data.loc[data['income_type'] == element, 'total_income'].fillna(median.loc[element, 'total_income'])

mean = data.groupby('income_type').agg({'days_employed' : 'mean'})
 
for element in data['income_type'].unique():
    data.loc[data['income_type'] == element, 'days_employed'] = data.loc[data['income_type'] == element, 'days_employed'].fillna(mean.loc[element, 'days_employed'])

display(data.head(40))
#Проверим замену
data.isnull().sum()
#Исследуем каждый столбец в отельности
#Посмотрим что с детьми
display(data['children'].unique())
# Значения -1 и 20 выглядят странно. Скорее всего, в значениях случайная ошибка при вводе, поэтому заменим -1 на 1, а значение 20 на 2
data['children'] = data['children'].abs()
data['children'] = data['children'].replace(20, 2)
# Кажется, что с детьми теперь порядок
display(data['children'].unique())
# Разберемся с возрастом
print(data['dob_years'].describe())
print(data['dob_years'].unique())
#Минимальный возраст 0, значит ошибка только в нем, так как все остальные возрасты >=19
#Заменим 0 возраст, на среднее значение в зависимости от типа зантости 
mean = data.groupby('income_type').agg({'dob_years' : 'mean'})

for element in data['income_type'].unique():
    data.loc[data['income_type'] == element, 'dob_years'] = data.loc[data['income_type'] == element, 'dob_years'].replace(0, mean.loc[element, 'dob_years'])
#Теперь с возрастом все в порядке, минимальный возраст 19 
display(data['dob_years'].describe())
# 'education' и 'family_status' в low-индекс
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['education'].unique()
# при приведении 'education', education_id тоже изменился

data['gender'].unique()
data[data['gender'] == 'XNA'].count()
display(data.head(20))




,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


array([ 1,  0,  3,  2, -1,  4, 20,  5])

array([1, 0, 3, 2, 4, 5])

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64
[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]


count    21525.00000
mean        43.49822
std         12.22943
min         19.00000
25%         34.00000
50%         43.00000
75%         53.00000
max         75.00000
Name: dob_years, dtype: float64

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Были расмотрены все столбцы нашей таблицы и устраненны проблемы
1. Столбец ['children'] - дети. Здесь значения -1 и 20 выглядят странно, Считаем, что это ошибка заменим -1 на 1, 20 на 2
2. Столбец ['days_employed'] - трудовой стаж в днях. Во-первых была решена проблема с отрицательными значениями. Во-вторых данный столбец содержал пропуски. Время — это количественная переменная. Пропуски в таких переменных заполняют характерными значениями. Чтобы примерно оценить типичные значения выборки, годится среднее арифметическое, так как стаж, если рассматривать его по группам примерно одинаков, тоесть нет сильно выделяющихся значений среди большинства. Заполним тогда значения с помощью расчета среднего для каждой группы занятости: для пенсионеров заполнять пропуски одним средним, студентам другим и тд. Все NaN по датафрейму заполнены, явно некорректные значения откорректированы. Возможно, что пропуски появились из-за того, что клиенты банка не указывали свой стаж. Так же в столбце ['days_employed'] огромные значения положительны скорее всего они указаны в часах, а не днях, поэтому стоит их поправить.
3. Столбец ['dob_years'] - возраст клиента в годах. Невозможен минимальный возраст 0, значит ошибка только в нем, так как все остальные возрасты >=19. Заменим 0 возраст, на среднее значение в зависимости от типа зантости.
4. Столбец ['education'] - образование клиента. Здесь были приведены строки к нижнему регистру, так как если не привести значения то они будут различны  и обращаться к ним будет сложно. Это исправляет автоматически и столбец ['education_id']
5. Столбец ['family_status'] - семейное положение. Произведем ту же  операцию, как и со столбцом ['education'], что исправляет и столбец ['family_status_id']
6. Столбец ['gender'] - пол клиента. Только одна запись с чем-то непонятным ('XNA'), но так как мы не знаем пол человека и не можем сделать предположений из данных, а также в расчетах использовать значение не будем, оставляем его как есть.
7. Столбцы ['income_type'] и ['debt'] содержат корректные значения.
8. Столбец ['total_income'] - доход в месяц. Данный столбец содержал пропуски. Среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства, так как зарплата может резко выделяться, у разных групп. Заполним тогда значения с помощью расчета медианы для каждой группы занятости: для пенсионеров заполнять пропуски одним средним, студентам другим и тд. Все NaN по датафрейму заполнены, явно некорректные значения откорректированы. Возможно, что так как не указывался стаж и соотвественно не подтягивались данные по месячному доходу.
9. Столбец ['purpose'] - цель получения кредита. Содержит корректные значения, но их нужно разбить на категории, так как цели похожие, а названия отличаются.


### Замена типа данных

In [17]:
# Смотреть на доходы с 6 знаками после запятой неудобно.
# Приведем к целочисленным значениям
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data['dob_years'] = data['dob_years'].astype('int')

display(data.dtypes)
display(data.head(20))

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Обработка дубликатов

In [28]:
# Посмотрим есть ли дубли
data.duplicated().sum()
# Удалим дубли и сбросим индекс
data = data.drop_duplicates().reset_index(drop=True)
# Проверим удалились ли дубли
data.duplicated().sum()


0

Был найден 71 дубликат, с помощью метода duplicated(). В сочетании с методом sum() он возвращает количество дубликатов.  Скорее всего это задвоение данных или ошибочное создание дублей.Необходимо их удалить, чтобы статистика не была искажена. Восстановить индексы помогает метод reset_index(). Датафрейм теперь без дубликатов.

### Лемматизация

In [18]:
# Узнаем уникальные значения
n = data['purpose'].unique().tolist()
m = Mystem()
string = '; '.join(n)
lemmas = m.lemmatize(string)
print(Counter(lemmas))

def lemmatizator(i):
    lemmas1 = m.lemmatize(i)
    return lemmas1
   
data['purpose_lemmas'] = data['purpose'].apply(lemmatizator)
 
display(data.tail(15))


Counter({' ': 59, '; ': 37, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
21510,2,2326,28,среднее,1,женат / замужем,0,F,сотрудник,0,142594,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
21511,0,612,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068,покупка жилья для сдачи,"[покупка, , жилье, , для, , сдача, \n]"
21512,0,165,26,высшее,0,не женат / не замужем,4,M,компаньон,0,147301,получение дополнительного образования,"[получение, , дополнительный, , образование,..."
21513,0,1166,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986,покупка жилья,"[покупка, , жилье, \n]"
21514,0,280,27,неоконченное высшее,2,не женат / не замужем,4,M,компаньон,0,355988,строительство недвижимости,"[строительство, , недвижимость, \n]"
21515,1,467,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486,заняться образованием,"[заниматься, , образование, \n]"
21516,0,914,42,высшее,0,женат / замужем,0,F,компаньон,0,322807,покупка своего жилья,"[покупка, , свой, , жилье, \n]"
21517,0,404,42,высшее,0,гражданский брак,1,F,компаньон,0,178059,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]"
21518,0,15583,59,среднее,1,женат / замужем,0,F,пенсионер,0,153864,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]"
21519,1,2351,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949,покупка коммерческой недвижимости,"[покупка, , коммерческий, , недвижимость, \n]"


Подсчёт числа упоминаний слов в тексте произвелось с помощью специального контейнера Counter. Здесь сразу стали видны необходимые категории. 
Леммитизация производилась в отдельном столбце, для того, чтобы была возможность категоризировать цели.
 
- автомобиль
- недвижимость
- образование
- свадьба

### Категоризация данных

In [19]:
def replace(i):
    if 'свадьба' in i:
        return 'свадьба'
    elif 'автомобиль' in i:
        return 'автомобиль'
    elif 'недвижимость' in i or 'жилье' in i:
        return 'недвижимость'
    elif 'образование' in i:
        return 'образование'

    
data['purpose_id'] = data['purpose_lemmas'].apply(replace)
data['purpose_id'].value_counts()


def replace_income(total_income):
    if total_income >= 20667 and total_income <= 64145:
        return 'низкий'
    if total_income > 64145 and total_income <= 107623:
        return 'средний'
    if total_income > 107623 and total_income <= 156044:
        return 'выше среднего'
    if total_income > 156044 and total_income <= 195813:
        return 'высокий'
    if total_income > 195813:
        return 'очень высокий'
    
    
data['total_income_id'] = data['total_income'].apply(replace_income)

#Категоризируем возраст 
def age(dob_years):
    if dob_years <= 39:
        return 'молодой'
    if 40 <= dob_years <= 59:
        return 'средний'
    if 60 <= dob_years <= 74:
        return 'пожилой'
    if dob_years >= 75:
        return 'старческий'
data['age'] = data['dob_years'].apply(age)
display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_id,total_income_id,age
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,очень высокий,средний
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,выше среднего,молодой
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,выше среднего,молодой
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,очень высокий,молодой
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,высокий,средний
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость,очень высокий,средний
21521,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]",автомобиль,выше среднего,пожилой
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]",недвижимость,средний,молодой
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]",автомобиль,очень высокий,молодой


Видим, что многие уникальные значения цели кредита названы похожим образом, но имеют одинаковый смысл. Мы лемматизируем их, чтобы затем разбить на категории. Категоризацию проводилась с соотвествием лемм по столбцу puprpose_lemmas. Выбрано 4 основные категории.
- автомобиль
- недвижимость
- образование
- свадьба
Также категоризации по столбцу total_income. 

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [20]:
data_children = pd.DataFrame()
data_children['count_children'] = data.groupby('children')['debt'].count()
data_children['sum_children'] = data.groupby('children')['debt'].sum()
data_children['result'] = data_children['sum_children'] / data_children['count_children'] * 100 
data_children.sort_values(by = 'result', ascending = False)


,count_children,sum_children,result
children,,,
4,41,4,9.756098
2,2131,202,9.479118
1,4865,445,9.146968
3,330,27,8.181818
0,14149,1063,7.512898
5,9,0,0.000000


Из данной таблицы можно сделать вывод, что клиенты без детей отвественнее всего подходят к выплате кредита в срок, впринципе, в этом есть логика, так как у них меньше затрат, им нужно тратиться только на себя. На втором месте расположились клиенты с 3-я детьми, возможно семьи с 3-я детьми более отвественны, чем семьи с 1-2 детьми. Так как у них есть большой опыт. На 3, 4 и 5 местах расположились семьи с 1-м, 2-я и 4-я детьм соотвественно. Скорее всего семьи с 1-м и 2-я детьми берут кредит одноразово, поэтому не так сильно ориентируются на срок выплат. Семьям с четырьмя детьми скорее всего не хватает денег, поэтому у них самые плохие показатели. Можно отметить, что люди с 5 детьми показывают 0, но выборка по ним не показательна, так как их мало.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [18]:
data_family = pd.DataFrame()
data_family['count_family'] = data.groupby('family_status')['debt'].count()
data_family['sum_family'] = data.groupby('family_status')['debt'].sum()
data_family['result'] = data_family['sum_family'] / data_family['count_family'] * 100 
data_family.sort_values(by = ['result'], ascending = False)


,count_family,sum_family,result
family_status,,,
не женат / не замужем,2810,274,9.750890
гражданский брак,4151,388,9.347145
женат / замужем,12339,931,7.545182
в разводе,1195,85,7.112971
вдовец / вдова,959,63,6.569343


 Как видно из таблицы, вдовец/вдова самые отвественные из клиентов. Люди не в браке и не бывавшие в браке имеют больший процент невозвратов в срок. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [19]:
data_total_income = pd.DataFrame()
data_total_income['count_total_income'] = data.groupby('total_income_id')['debt'].count()
data_total_income['sum_total_income'] = data.groupby('total_income_id')['debt'].sum()
data_total_income['result'] = data_total_income['sum_total_income'] / data_total_income['count_total_income'] * 100 
data_total_income.sort_values(by = ['result'], ascending = False)

,count_total_income,sum_total_income,result
total_income_id,,,
выше среднего,6964,608,8.730615
высокий,3761,323,8.588141
средний,4323,358,8.281286
очень высокий,5365,383,7.138863
низкий,1041,69,6.628242


Лучший средний показатель имеют клиенты с низким доходом. Рассуждая логически, клиенты с низким доходом понимают, что в им нужна положительная кредитная история, для того, чтобы одобрили следующий. Клиенты с очень высоким доходом вполне возможно берут кредиты как предприниматели, чтобы взять кредит для бизнеса, поэтому тоже не хотят портить кредитную историю. Самый худший показатель у клиентов с доходом выше среднего, скорее всего эти люди берут кредиты и забывают их возвращать 

- Как разные цели кредита влияют на его возврат в срок?

In [20]:
data_purpose = pd.DataFrame()
data_purpose['count_purpose'] = data.groupby('purpose_id')['debt'].count()
data_purpose['sum_purpose'] = data.groupby('purpose_id')['debt'].sum()
data_purpose['result'] = data_purpose['sum_purpose'] / data_purpose['count_purpose'] * 100 
data_purpose.sort_values(by = ['result'], ascending = False)

,count_purpose,sum_purpose,result
purpose_id,,,
автомобиль,4306,403,9.359034
образование,4013,370,9.220035
свадьба,2324,186,8.003442
недвижимость,10811,782,7.233373


Луший показатель у клиентов с целью покупки недвижимости. Думаю это связанно с тем что это одна из самых ответственных и дорогих покупок у клиентов, поэтому они стремятся выплатить все вовремя. Разница между минимальным и максимальными средними значениями равна 23%

## Шаг 4. Общий вывод

Семейное положение и количество детей влияет на факт погашения кредита в строк:
- Заемщики с официально оформленными отношениями (или которые в прошлом были в официальном в браке) и не имеющие детей - самые ответственные заемщики;
- Заемщики, состоящие в неофициальном браке или находящиеся без отношений, при этом имеющие 1 или 2 детей - самые менее ответственные заемщики
В целом, по исследованным данным можно сделать следующий вывод:

Портрет идеального клиента: 
- Не имеет детей	
- Вдовец/вдова
- Уровень дохода < 20667
- Берет кредит на недвижимость

Портрет потенциального должника:
- Имеет 1-2 детей
- Люди не в браке и не бывавшие в браке
- Уровень дохода [107623, 56044]	
- Берет кредит на автомобиль/образование